In [8]:
# EM RELAÇÃO A CAPTURA DOS DADOS SOMENTES AS 3 PRIMEIRAS CÉLULAS DESTE NOTEBOOKS SÃO RELEVANTES
# O RESTO É ANÁLISES DE GRÁFICOS

import pandas as pd
import urllib.request

#-------------------------------------------------------------------------------
from google.colab import drive # não é necessário de não for rodar no google colab (comentar linha)
drive.mount('/content/drive')  # não é necessário de não for rodar no google colab (comentar linha)
# Local de salvamento dos arquivos, modificar conforme necessidade
path = r"/content/drive/My Drive/Colab Notebooks/covid19/raspagem de dados/" 
#-------------------------------------------------------------------------------

# PREPARAÇÃO DO DATAFRAME DE REFERÊNCIA 

def load_data(imp_ind = True):
    url = "https://brasil.io/dataset/covid19/caso?format=csv"
    df = pd.read_csv(url)
    df = df.fillna(0)
    if imp_ind == False:
        df = df.loc[df["city"] != "Importados/Indefinidos"] # retira os cados importados/indefinidos caso false
    else:
        pass
    return df

# teste de requisição dos dados
df = load_data(imp_ind = False)
# df = df.loc[df["date"] == "2020-03-30"]
# df = df.loc[df["place_type"] == "city"]
# df = df.loc[df["city"] == "Importados/Indefinidos"]
len(df)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


3143

In [10]:
# ADICIONANDO LATITUDE E LONGITUDE NO DATASET E SALVAMENTO DO MESMO

!pip install geocoder
import geocoder # biblioteca geocoder (neste caso específico utiliza a API do bing maps)

# esse token é pessoal, mas pode ser usado para testes, em caso de aplicação deve ser gerado um novo
key = 'AiXNSTykHCrpomqs090zP_tNFDTuLzim-yg78FJ4xbUKOnFJ7EAUmjIAVN8QGQ1d' 

# Carrega todo o dataset
df_fullData = load_data(imp_ind = False) # Importados/Indefinidos == False

# Vetor lagitude e longitude que será aplicado como colunas no dataframa
latlong = []

# Dicionário para armazenamento temporário de cidades/estados já analisados (evitar retrabalho do geocoder)
cidEst = {"cid_est":[], "lat":[], "lng":[]}

for index, row in df_fullData.iterrows():

	if row["place_type"] == "state":
		cidest = row["state"]
	elif row["place_type"] == "city":
		cidest = row["city"]
		
	if cidest in cidEst["cid_est"]:
		lat = cidEst["lat"][cidEst["cid_est"].index(cidest)]
		lng = cidEst["lng"][cidEst["cid_est"].index(cidest)]
	else:
		loc = None
		while loc is None:
			try:
				if row["place_type"] == "state":
					loc = geocoder.bing(str(cidest) + ', ' + "BR", key=key)
				elif row["place_type"] == "city":
					loc = geocoder.bing(str(cidest)+', '+str(row['state'])+', '+"BR", key=key)
				lat = loc.json["lat"]
				lng = loc.json["lng"]
				cidEst["lat"] = lat
				cidEst["lng"] = lng
				# print(f"{cidest}: {lat, lng}")
			except:
				print(f"{cidest}: CIDADE/ESTADO NÃO ENCONTRADO")
				print("Tentando novamente!")		
		# print("Deu tudo certo!")

	latlong.append([lat, lng])

print(len(latlong))

df_latLong_full = pd.DataFrame(latlong, columns=['latitude', 'longitude'], index=df_fullData.index)

df_fullData_new = pd.concat([df_fullData, df_latLong_full], axis=1)

df_fullData_new.to_csv(path+"fullData.csv", index=False) # o caminho aqui deve ser alterado caso necessário

df_fullData_new.head()

3143


,date,state,city,place_type,confirmed,deaths,is_last,estimated_population_2019,city_ibge_code,confirmed_per_100k_inhabitants,death_rate,latitude,longitude
0,2020-03-31,AP,Macapá,city,10,0.0,True,503327.0,1600303.0,1.98678,0.0000,0.034080,-51.066490
1,2020-03-31,AP,0,state,10,0.0,True,845731.0,16.0,1.18241,0.0000,0.036410,-51.048290
2,2020-03-31,MG,Alfenas,city,1,0.0,True,79996.0,3101607.0,1.25006,0.0000,-21.436090,-45.944809
3,2020-03-31,MG,Belo Horizonte,city,163,2.0,True,2512070.0,3106200.0,6.48867,0.0123,-19.919519,-43.938332
4,2020-03-31,MG,Betim,city,5,0.0,True,439340.0,3106705.0,1.13807,0.0000,-19.967649,-44.199249


In [11]:
# TESTE DE USO DA BIBLIOTECA google_drive_downloader
!pip install googledrivedownloader

def gdownloader(): # baixa os dados do google drive 
    from google_drive_downloader import GoogleDriveDownloader as gdd 
    ID = "1ji4P3Gjpbc4LpbDigLqPcZM1t8Bvekh-" # id do arquivo 
    dest_path = './data.csv' 
    gdd.download_file_from_google_drive( file_id = ID, dest_path = dest_path ) 
    df = pd.read_csv(dest_path) 
    return df

df = gdownloader() 
df.head()

,date,state,city,place_type,confirmed,deaths,is_last,estimated_population_2019,city_ibge_code,confirmed_per_100k_inhabitants,death_rate,latitude,longitude
0,2020-03-31,AP,Macapá,city,10,0.0,True,503327.0,1600303.0,1.98678,0.0000,0.034080,-51.066490
1,2020-03-31,AP,0,state,10,0.0,True,845731.0,16.0,1.18241,0.0000,0.036410,-51.048290
2,2020-03-31,MG,Alfenas,city,1,0.0,True,79996.0,3101607.0,1.25006,0.0000,-21.436090,-45.944809
3,2020-03-31,MG,Belo Horizonte,city,163,2.0,True,2512070.0,3106200.0,6.48867,0.0123,-19.919519,-43.938332
4,2020-03-31,MG,Betim,city,5,0.0,True,439340.0,3106705.0,1.13807,0.0000,-19.967649,-44.199249


In [4]:
# Análise descritiva dos dados

df = pd.read_csv(path+"fullData.csv", encoding='utf-8')

df.describe()

,confirmed,deaths,estimated_population_2019,city_ibge_code,confirmed_per_100k_inhabitants,death_rate,latitude,longitude
count,3052.000000,3052.000000,3.052000e+03,3.052000e+03,3052.000000,3052.000000,3052.000000,3052.000000
mean,18.771953,0.431193,1.880537e+06,3.025362e+06,3.075018,0.014150,-19.164738,-46.836105
std,85.058206,4.626338,5.604522e+06,1.521114e+06,6.558234,0.097596,8.681459,6.584173
min,0.000000,0.000000,2.878000e+03,1.100000e+01,0.000000,0.000000,-32.033138,-67.946953
25%,1.000000,0.000000,7.008400e+04,2.408102e+06,0.475500,0.000000,-26.350229,-51.086651
50%,2.000000,0.000000,2.316710e+05,3.301875e+06,1.223480,0.000000,-20.997030,-47.928650
75%,6.000000,0.000000,8.090150e+05,4.209102e+06,2.989605,0.000000,-12.255520,-42.536400
max,1517.000000,113.000000,4.591905e+07,5.300108e+06,69.492700,1.000000,3.361190,-32.408669


In [27]:
# PLOTAR GRÁFICO DE PIZZA ()

def viz(date, place_type, place):
    df1 = pd.read_csv(path+"fullData.csv", encoding='utf-8')
    df2 = df1.loc[df1["date"      ] == date      ]
    df3 = df2.loc[df2["place_type"] == place_type]
    df4 = df3.loc[df3[place_type  ] == place     ]
    return df4

estado = "SP"
df = viz("2020-03-30", "state", estado)

import plotly.graph_objects as go

labels = ["Confirmados", "Mortes"]
values = [int(df["confirmed"]), int(df["deaths"])]

pizza = go.Pie( labels = labels, values = values, pull = [0, .2])

fig = go.Figure( data = [ pizza ] )

fig.update_layout( title_text = "Gráfico de pizza para o estado: "+str(estado) )

fig.show()
df

,date,state,city,place_type,confirmed,deaths,is_last,estimated_population_2019,city_ibge_code,confirmed_per_100k_inhabitants,death_rate,latitude,longitude
393,2020-03-30,SP,0,state,1517,113.0,True,45919049.0,35.0,3.30364,0.0745,-23.53397,-46.628868


In [25]:
import plotly.graph_objects as go

fig = go.Figure()

df = pd.read_csv(path+"fullData.csv", encoding='utf-8')
df = df.loc[ : , ["date", "state", "city", "place_type", "confirmed", "deaths"]]
df = df.loc[df["place_type"] == "state"]
# df = df.loc[df["date"] == "2020-03-28"]
df = df.sort_values(by="date")

datas = []

for data in df["date"]:
    if data not in datas:
        datas.append(data)

confirmados = []
mortes = []

for i in range(0, len(datas)):
    confirmados.append(sum(df["confirmed"].loc[df["date"] == datas[i]]))
    mortes.append(sum(df["deaths"].loc[df["date"] == datas[i]]))


df = pd.DataFrame({"datas":datas, "confirmados":confirmados, "mortes":mortes})

# df.plot()

fig.add_trace(go.Bar(
    x = df["datas"],
    y = df["confirmados"],
    name = 'Casos confirmados acumulado',
    marker_color = 'indianred'
))

fig.add_trace(go.Bar(
    x = df["datas"],
    y = df["mortes"],
    name='Mortes acumuladas',
    marker_color='lightsalmon'
))

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='group', xaxis_tickangle=-45)
fig.show()

# df

In [9]:
import folium
from folium import plugins
import geocoder

key = 'AiXNSTykHCrpomqs090zP_tNFDTuLzim-yg78FJ4xbUKOnFJ7EAUmjIAVN8QGQ1d'

df = pd.read_csv(path+"fullData.csv", encoding='utf-8')

# data = viz("2020-03-25", "SP", "PR")

data = df

loc = geocoder.bing("BR", key=key)
lat = loc.json["lat"]
lng = loc.json["lng"]

brasil = folium.Map(
    location   = [lat, lng], # https://www.geodatos.net/en/coordinates/brazil
    zoom_start = 4
)

latlong = []
for _, rows in data.iterrows():
    latlong.append([rows["latitude"], rows["longitude"]])

latlong = [[data["latitude"].iloc[i], data["longitude"].iloc[i]] for i in range(0, len(data["latitude"]))]

HM = plugins.HeatMap(
    latlong, 
    radius      = 8, 
    min_opacity = 1, 
    max_val     = 1,
    gradient    = {.2: 'blue', .6: 'lime', 1: 'red'}
)

HM.add_to(brasil)

brasil.save(outfile= "test.html")

brasil

In [31]:
import plotly.express as px
import plotly
import pandas as pd

adm = "state"

df = pd.read_csv(path+"fullData.csv", encoding='utf-8')
df1 = df[df["place_type"] == adm]
df1 = df.sort_values(by = "date")

param = "confirmed" 

fig = px.scatter_geo(
    df1, 
    lat = "latitude", 
    lon = "longitude", 
    color = param,
    hover_name = adm, 
    size = param,
    animation_frame = "date",
    projection = "natural earth",
)

fig.update_layout(
    title = 'Gráfico de bolhas - Demonstração de casos confirmados ao longo do tempo',
    geo_scope = 'south america'
) 

fig.show()

plotly.offline.plot(
    fig, 
    filename='covid19_BR_animation.html'
)

'covid19_BR_animation.html'